In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from nltk.translate.bleu_score import sentence_bleu
from transformers import DistilBertModel,DistilBertTokenizer
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm

/Users/adam/PycharmProjects/SOTA/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
PATH_ROOT = "/Users/adam/PycharmProjects/SOTA"
%cd {PATH_ROOT}

/Users/adam/PycharmProjects/SOTA


In [3]:
df = pd.read_csv(os.path.join(PATH_ROOT,"data/combined/with_annotators/combined_dataset.csv"), index_col=0)

In [4]:
df.head(3)

,annotator,text1,text2,label,dataset,random,duration,total_seconds,pair_id,reduced_label,...,bleu1,glove_cosine,fasttext_cosine,BertScore,chrfScore,POS Dist score,1-gram_overlap,ROUGE-1,ROUGE-2,ROUGE-l
0,A3BCGN11HDM8QR,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",2,bible_human,0,0 days 00:00:12.000000000,12,pair_0,-1,...,0.777466,0.015252,0.035903,0.969967,0.699194,1.232655,0.612903,0.782609,0.626866,0.75
1,A3SQ00HYQN7FYB,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",3,bible_human,0,0 days 00:00:12.000000000,12,pair_0,0,...,0.777466,0.015252,0.035903,0.969967,0.699194,1.232655,0.612903,0.782609,0.626866,0.75
2,A5WAWW70PYRP,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",4,bible_human,0,0 days 00:07:19.000000000,439,pair_0,1,...,0.777466,0.015252,0.035903,0.969967,0.699194,1.232655,0.612903,0.782609,0.626866,0.75


In [5]:
# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:
# 
# 1. Find out who the bad actors are
# 2. See if there is any overlap from the dif. category of bad actors
# 
# (Removing bad phrases should also be done - but is beyond the scope of this notebook)
# 
# 
# The categories of bad actors are:
# 
# [x] - Slow Annotators - ba_time </br>
# [x] - Greater variance in random datasets than non-random datasets - ba_unvar_annotations </br>
# [x] - Unpopular (disagree with two others often) - ba_unpopular </br>
# [ ] - Inconsistent with sentiment != semantics - ba_semantics </br>

# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers

Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:

1. Find out who the bad actors are
2. See if there is any overlap from the dif. category of bad actors

(Removing bad phrases should also be done - but is beyond the scope of this notebook)


The categories of bad actors are:

[x] - Slow Annotators </br>
[x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
[ ] - Inconsistent with sentiment != semantics </br>

## Time Outliers

Under the assumption that anyone that takes over the 95 percentile of time.

In [6]:
print(df.total_seconds.describe(percentiles = [.25,.5,.75,.9,.95]))

# ba = bad actor
df['mean_annotation_time'] = df.groupby('annotator').total_seconds.transform('mean')
print(df.mean_annotation_time.describe(percentiles = [.25,.5,.75,.9,.95]))
ba_time = df[df.mean_annotation_time > 405].annotator.unique().tolist()
print(len(df[df.total_seconds > 336].annotator.unique().tolist()))
print(len(ba_time))

count    39660.000000
mean        86.010867
std        427.060791
min          1.000000
25%          7.000000
50%         12.000000
75%         28.000000
90%        169.000000
95%        336.000000
max      14079.000000
Name: total_seconds, dtype: float64
count    39660.000000
mean        86.010867
std        320.021426
min          3.146341
25%          9.910026
50%         17.632653
75%         50.928105
90%        120.605431
95%        405.681159
max      13375.500000
Name: mean_annotation_time, dtype: float64
110
31


## Unvarianced Annotations
Labelers whos std is too low mean non-random - random difference is too high  

In [7]:
labelers = df[df.random==0].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers = labelers[labelers['size']>1]
#df = df[df.annotator.apply(lambda x:x in set(labelers.index))]

labelers_rand = df[df.random==1].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers_rand = labelers_rand[labelers_rand['size']>1]
labelers = labelers.join(labelers_rand, rsuffix = '_rand')
labelers['mean_random_gap'] = labelers['mean']-labelers['mean_rand']
labelers['std_ratio'] = labelers['std']/labelers['std_rand']

total_std = df.groupby('annotator')['label'].std()
total_std.name = 'total_std'
labelers = labelers.join(total_std)

In [8]:
ba_unvar_annotations = labelers[(labelers.total_std<1) & (labelers.mean_random_gap < 0)].index.tolist()
len(ba_unvar_annotations)

19

## Unpopular Annotators
Those who over 50% of the time, disagree with the other annotators (in the reduced label)

In [9]:
df_uniquelabels = df.groupby("pair_id")["reduced_label"].nunique()
pairs_twoagree = df_uniquelabels[(df.groupby("pair_id")["reduced_label"].nunique() == 2).values].index.tolist()
df_twoagree = df[df["pair_id"].isin(pairs_twoagree)]

df_twoagree['generally_accepted_label'] = df_twoagree.groupby("pair_id")['reduced_label'].transform('median')

<ipython-input-9-9829361a5538>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_twoagree['generally_accepted_label'] = df_twoagree.groupby("pair_id")['reduced_label'].transform('median')


In [10]:
df_twoagree.head()

,annotator,text1,text2,label,dataset,random,duration,total_seconds,pair_id,reduced_label,...,fasttext_cosine,BertScore,chrfScore,POS Dist score,1-gram_overlap,ROUGE-1,ROUGE-2,ROUGE-l,mean_annotation_time,generally_accepted_label
6,A10DMENL2H243H,"And the lord of that servant , being moved wit...","And the lord of that servant , being moved wit...",3,bible_human,0,0 days 00:00:12.000000000,12,pair_2,0,...,0.040671,0.969194,0.607663,1.403221,0.636364,0.790698,0.536585,0.764706,12.000000,1
7,A2G44A4ZPWRPXU,"And the lord of that servant , being moved wit...","And the lord of that servant , being moved wit...",5,bible_human,0,0 days 00:08:20.000000000,500,pair_2,1,...,0.040671,0.969194,0.607663,1.403221,0.636364,0.790698,0.536585,0.764706,1263.424242,1
8,ACKF1WBCKVLY3,"And the lord of that servant , being moved wit...","And the lord of that servant , being moved wit...",4,bible_human,0,0 days 00:00:06.000000000,6,pair_2,1,...,0.040671,0.969194,0.607663,1.403221,0.636364,0.790698,0.536585,0.764706,9.584450,1
15,ANS8B7J9GNA2W,having become by so much better than the angel...,taking a place by so much better than the ange...,4,bible_human,0,0 days 00:00:58.000000000,58,pair_5,1,...,0.041350,0.932658,0.682740,0.000000,0.695652,0.800000,0.578947,0.756757,3054.882353,1
16,AE7UECBR4D5E6,having become by so much better than the angel...,taking a place by so much better than the ange...,3,bible_human,0,0 days 00:00:14.000000000,14,pair_5,0,...,0.041350,0.932658,0.682740,0.000000,0.695652,0.800000,0.578947,0.756757,21.127424,1


In [11]:
df_unpopularopinion = df_twoagree[df_twoagree.reduced_label != df_twoagree.generally_accepted_label].groupby('annotator').size().reset_index()
df_unpopularopinion.columns = ['annotator','unpopular_opinion']

df_allopinions = df[df['annotator'].isin(list(df_unpopularopinion.annotator))].groupby('annotator').size().reset_index()
df_allopinions.columns = ['annotator','all_opinion']

df_opinion_all_unpop = df_allopinions.merge(df_unpopularopinion,on="annotator")

In [12]:
df_opinion_all_unpop.head()

,annotator,all_opinion,unpopular_opinion
0,A10DMENL2H243H,1,1
1,A110SC5K5Y3IHS,5,1
2,A11YCM5MVQ35YY,33,1
3,A123PBQDU71I5O,7,3
4,A12BPQY35PARUO,207,23


In [13]:
ba_unpopular = df_opinion_all_unpop[((df_opinion_all_unpop.unpopular_opinion / df_opinion_all_unpop.all_opinion) > 0.5) & (df_opinion_all_unpop.all_opinion > 4)].annotator.tolist()

## Sentiment // Semantic Understanding

In [14]:
# Sentiment analysis pipeline
sentiment_pipe = pipeline("sentiment-analysis")

In [15]:
from tqdm import tqdm

text1_sent,text2_sent =[], []

pbar = tqdm(total = len(df)//100+1, position = 0, leave = True)
for i in range (len(df)//100+1):
    t1_s = sentiment_pipe(df.text1.tolist()[100*i:np.min([100*i+100,len(df)])])
    t2_s = sentiment_pipe(df.text2.tolist()[100*i:np.min([100*i+100,len(df)])])
    text1_sent+=t1_s
    text2_sent+=t2_s
    pbar.update()
pbar.close()
len(text1_sent)

100%|██████████| 397/397 [23:26<00:00,  3.54s/it]


39660

In [ ]:
sent = {'POSITIVE':1,'NEGATIVE':-1}

df['sentiment_1'] = np.array([x['score']*sent[x['label']] for x in text1_sent]) 
df['sentiment_2'] = np.array([x['score']*sent[x['label']] for x in text2_sent])
df['dif_sent'] =  np.abs(df['sentiment_1']-df['sentiment_2'])

In [ ]:
pairs = []
for index, row in df.iterrows():
    first_sentence_tokens = row['text1'].strip().split()
    second_sentence_tokens = row['text2'].strip().split()
    pairs.append((first_sentence_tokens, second_sentence_tokens))

In [ ]:
scores_bleu1 = []
for first_sentence_tokens, second_sentence_tokens in pairs:

    score_bleu1 = sentence_bleu([first_sentence_tokens], second_sentence_tokens, weights=(1, 0, 0, 0))
    scores_bleu1.append(score_bleu1)

print(np.mean(scores_bleu1))
print(np.std(scores_bleu1))


In [ ]:
df['bleu_score_1'] = scores_bleu1

In [ ]:
annot_std_semantic = df[(df['bleu_score_1'] > 0.8) & (df['dif_sent'] > 1.9)].groupby('annotator')['label'].std().dropna()
ba_semantics = list(annot_std_semantic[annot_std_semantic > 1.0].index)

# Combining all the results

In [ ]:
import itertools

In [ ]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [ ]:
all_ba = ['ba_semantics','ba_time','ba_unpopular','ba_unvar_annotations']

for a,b in list(itertools.combinations(all_ba,2)):
    print(f"Jaccard Similarity of {a} and {b} is :{jaccard_similarity(eval(a),eval(b))}")

While the overlap isn't super consistent, it is interesting to note that the two most correlated groups are time and unpopularity and unpopularity with unvaried annotations.

In [ ]:
all_ba = list(set(ba_unvar_annotations + ba_unpopular + ba_time + ba_semantics))
print(f"Total number of bad annotators are: {len(all_ba)}")
print(f"Percentage of total annotators are: {len(all_ba)/df.annotator.nunique()}")

### Save the annotators so we can filter them out quicker later

In [ ]:
for ba in all_ba:
    with open(f'data/other/{ba}.txt', 'w') as f:
        for item in eval(ba):
            f.write("%s\n" % item)

In [ ]:
with open('data/other/ba_all.txt','w') as f:
    for item in list(set(ba_unvar_annotations + ba_unpopular + ba_time + ba_semantics)):
        f.write("%s\n" % item)